In [ ]:
!pip install -q llama-index-llms-openai llama-index ragas

In [2]:
import os
from llama_index.core import SimpleDirectoryReader
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import VectorStoreIndex
from llama_index.core.settings import Settings

In [3]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple,reasoning,multi_context

from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
    answer_correctness,
)

In [4]:
from ragas.metrics.critique import harmfulness
from ragas.integrations.llama_index import evaluate

In [5]:
from google.colab import userdata

os.environ['OPENAI_API_KEY']="YOUR KEY"

In [6]:
embeddings=OpenAIEmbedding()
Settings.embed_model=embeddings

In [8]:
generator_llm=OpenAI(model="gpt-4o-mini")
critics_llm=OpenAI(model="gpt-4o-mini")
evalutor_llm=OpenAI(model="gpt-4o-mini")

PATH="./STORE"

In [ ]:
document=SimpleDirectoryReader(input_files=[r"/content/gita.pdf"]).load_data()

generator=TestsetGenerator.from_llama_index(
    generator_llm=generator_llm,
    critic_llm=critics_llm,
    embeddings=embeddings,
)

testset=generator.generate_with_llamaindex_docs(
    documents=document,
    test_size=14,
    distributions={simple:0.25, reasoning:0.5, multi_context:0.25},
)

In [10]:
df=testset.to_pandas()
df.head()

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What does the concept of sahajam karma imply a...,[Sahajam karma kaunteya sadoshamapi na tyajet;...,The concept of sahajam karma implies that one ...,simple,"[{'page_label': '150', 'file_name': 'gita.pdf'...",True
1,What is the relationship between liberation an...,[Yatendriya manobuddhir munir mokshaparaayanah...,The Bhagavad Gita describes that liberation is...,simple,"[{'page_label': '69', 'file_name': 'gita.pdf',...",True
2,What is the significance of the purification o...,[9. He who is of the same mind to the good-hea...,The answer to given question is not present in...,simple,"[{'page_label': '72', 'file_name': 'gita.pdf',...",True
3,What role does Lord Krishna play in the speake...,[liberation while living and disembodied liber...,Lord Krishna is portrayed as the indweller of ...,simple,"[{'page_label': '20', 'file_name': 'gita.pdf',...",True
4,What ties link Dhritarashtra's kids and Pandu'...,[INTRODUCTION\nDhritarashtra and Pandu were br...,"Dhritarashtra's children, the Kauravas, and Pa...",reasoning,"[{'page_label': '17', 'file_name': 'gita.pdf',...",True


In [12]:
ds=testset.to_dataset()
ds

Dataset({
    features: ['question', 'contexts', 'ground_truth', 'evolution_type', 'metadata', 'episode_done'],
    num_rows: 14
})

In [11]:
vector_index=VectorStoreIndex.from_documents(documents=document)
vector_index.storage_context.persist(persist_dir=PATH)

In [12]:
query_engine = vector_index.as_query_engine()

In [47]:
questions = testset.to_pandas()["question"].to_list()
ground_truth = testset.to_pandas()["ground_truth"].to_list()
contexts = testset.to_pandas()["contexts"].to_list()

In [ ]:
from datasets import Dataset

data = {
    'question': [],
    'ground_truth': ground_truth,
    'answer': [],
    'contexts': contexts,
}

print(questions[0])
print(ground_truth[0])
print(contexts[0])

In [51]:

for item in questions:
    data['question'].append(item)
    data['answer'].append(query_engine.query(item).response)
    # data['contexts'].append([query_engine.query(item).source_nodes[1].node.get_text()])

In [ ]:
print(data)

In [55]:
dataset = Dataset.from_dict(data)

In [57]:
metrics=[
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
    harmfulness,
    answer_correctness,
]

result=evaluate(
      query_engine=query_engine,
      metrics=metrics,
      dataset=dataset,
      # llm=evalutor_llm,
      # embeddings=embeddings,
      raise_exceptions=False,
)

#dataset['answer'] is done

Running Query Engine:   0%|          | 0/12 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/72 [00:00<?, ?it/s]

In [60]:
ans = result.to_pandas()
ans.head()

,question,contexts,answer,ground_truth,faithfulness,answer_relevancy,context_precision,context_recall,harmfulness,answer_correctness
0,What does the concept of sahajam karma imply a...,[Yajno daanam tapashchaiva paavanaani maneeshi...,The concept of sahajam karma implies that one ...,The concept of sahajam karma implies that one ...,1.0,0.954665,1.0,0.5,0,0.731243
1,What is the relationship between liberation an...,[Yatendriya manobuddhir munir mokshaparaayanah...,Liberation in the Bhagavad Gita is described a...,The Bhagavad Gita describes that liberation is...,1.0,0.944619,1.0,1.0,0,0.994035
2,What is the significance of the purification o...,[erect but not tensed. He should fix his purif...,The purification of the self in the practice o...,The answer to given question is not present in...,1.0,0.999880,0.0,1.0,1,0.926233
3,What role does Lord Krishna play in the speake...,[IV\nTHE YOGA OF WISDOM\nSummary of Fourth Dis...,Lord Krishna plays a significant role in the s...,Lord Krishna is portrayed as the indweller of ...,1.0,0.991542,1.0,1.0,1,0.993300
4,What ties link Dhritarashtra's kids and Pandu'...,[INTRODUCTION\nDhritarashtra and Pandu were br...,Dhritarashtra's kids and Pandu's sons were bro...,"Dhritarashtra's children, the Kauravas, and Pa...",1.0,0.928323,1.0,1.0,0,0.574910
